# Computes inter-agreement (inter-annotators agreement)

## Libs versions

In [1]:
import sys

# (Tested with 3.6)
print(f'{sys.version}')

3.6.4 |Anaconda custom (64-bit)| (default, Dec 21 2017, 21:42:08) 
[GCC 7.2.0]


In [2]:
import os
import pathlib
import csv as csv
import pandas as pd
import numpy as np
import sklearn as sk
import itertools
import functools
import sklearn.metrics as skmetrics
from sklearn.metrics import cohen_kappa_score
from sklearn import preprocessing

# Enable inline plotting
%matplotlib inline

In [3]:
pd.__version__

'0.22.0'

In [4]:
np.__version__

'1.12.1'

In [5]:
sk.__version__

'0.19.1'

## Set display options (for the Jupyter notebook)

In [6]:
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 100

## load the CSV

In [7]:
#dir_base = '/home/ojeulin/developpement/data/IA-et-droit-zonage-décisions-lots/lot-brut'
#file_csv_annotations = pathlib.Path(dir_base) / 'annotations-full.csv'
dir_base = '/home/ojeulin/developpement/data/IA-et-droit-zonage-décisions-lots/brat_6-12'
file_csv_annotations = pathlib.Path(dir_base) / 'annotations-clean.csv'

csv_separator_out = ','
csv_encoding = 'utf-8'

In [8]:
print(f'data file: {file_csv_annotations}')

data file: /home/ojeulin/developpement/data/IA-et-droit-zonage-décisions-lots/brat_6-12/annotations-clean.csv


In [9]:
df = pd.read_table(file_csv_annotations
        , sep=csv_separator_out
        , encoding=csv_encoding
        )

In [10]:
df.head()

,batch,file,total_line_number,line_num,types,annotation_difficulty,types_macro,sub_type,Parties,Argument,text
0,lot_0037,JURITEXT000033216500.txt,177,1,n_a,Difficile,Entete,n_a,n_a,NaN,COUR D'APPEL DE VERSAILLES
1,lot_0037,JURITEXT000033216500.txt,177,2,n_a,Difficile,Entete,n_a,n_a,NaN,Code nac : 53B
2,lot_0037,JURITEXT000033216500.txt,177,3,n_a,Difficile,Entete,n_a,n_a,NaN,16e chambre
3,lot_0037,JURITEXT000033216500.txt,177,4,n_a,Difficile,Entete,n_a,n_a,NaN,ARRET No
4,lot_0037,JURITEXT000033216500.txt,177,5,n_a,Difficile,Entete,n_a,n_a,NaN,CONTRADICTOIRE


### Consistency check

In [11]:
def check_csv_lines(file_csv, df):
    wc = !wc -l "$file_csv"
    line_count_csv = int(wc[0].split(' ')[0]) - 1 # skip 1st line, the header
    line_count_df = df.shape[0]
    assert line_count_csv == line_count_df, f'Error: number of lines in the CSV file (#{line_count_csv}) ≠ number of lines in the DataFrame (#{line_count_df})'

In [12]:
check_csv_lines(file_csv_annotations, df)

## Pre-process data

### Get the filename and directory used for inter-annotator agreement
These are the files existing in more than 1 directory.

In [13]:
df_grouped = df[['file', 'batch']].drop_duplicates(['file', 'batch']).groupby(['file']).filter(lambda group: len(group) > 1)

In [14]:
df['file_interagr'] = df_grouped.file

Check a file in 2 batchs:


In [15]:
f_i = df_grouped.describe().loc['top'].file

In [16]:
df[df.file == f_i][pd.notnull(df[df.file == f_i].file_interagr)]

,batch,file,total_line_number,line_num,types,annotation_difficulty,types_macro,sub_type,Parties,Argument,text,file_interagr
16309,lot_0011,JURITEXT000033184854.txt,95,1,n_a,Moyen,Entete,n_a,n_a,NaN,Ch. civile A,JURITEXT000033184854.txt
16404,lot_0012,JURITEXT000033184854.txt,95,1,n_a,Facile,Entete,n_a,n_a,NaN,Ch. civile A,JURITEXT000033184854.txt


### Extract lines for files used for inter-agreement

In [17]:
df_interagr = df.query('file in file_interagr').copy()

In [18]:
df_interagr.drop('file_interagr', axis=1, inplace=True)

### Drop lines having type = 'n_a' for a given (file, line_num)
Those lines are almost always correct and aren't usefull for the Cohen κ.

In [19]:
df_interagr.shape

(17106, 11)

In [20]:
def types_all_na(g):
    return (g.types == 'n_a').all()    

to_drop = df_interagr.groupby(['file', 'line_num']).filter(types_all_na)
to_drop.shape

(3786, 11)

In [21]:
df_interagr.drop(to_drop.index, inplace=True)

In [22]:
df_interagr.shape

(13320, 11)

### Get annotators names for each directory ('Lot')
This CSV is generated from the Google drive used to trach users and their working directories.

In [23]:
file_csv_users = pathlib.Path(dir_base) / 'Annotateurs sur le projet zonage - Attribution lots.csv'
df_users = pd.read_table(file_csv_users
                         , sep=','
                         , encoding=csv_encoding
                        )

In [24]:
check_csv_lines(file_csv_users, df_users)

In [25]:
# rename columns
# (note: name = 'nom'; batch = 'lot')
df_users.rename(columns={"Nom/pseudo de l'utilisateur": 'nom', "Lot ": 'lot'}, inplace=True)

In [26]:
df_users

,Login,Password,nom,lot,Etat de l'annotation du lot en cours
0,user1,user1,Camille Le Douaron,Lot 1,Finie
1,user2,user2,Eve Donnadieu,Lot 2,Finie
2,user3,user3,Isabelle Francelet,Lot 3,Finie
3,user4,user4,Camille Le Douaron,Lot 4,Finie
4,user5,user5,Céline Baudon,Lot 5,Finie
5,user6,user6,Marie-Laure Bellamy-Brown,Lot 6,Finie
6,user7,user7,Eve Donnadieu,Lot 7,Finie
7,user8,user8,Camille Le Douaron,Lot 8,Finie
8,user9,user9,Céline Baudon,Lot 9,Finie
9,user10,user10,Camille Le Douaron,Lot 10,Finie


In [27]:
%%time

df_batchs = df_users[['nom', 'lot']]

def name_from_batch(batch):
    """Lot 14 → John Doe"""
    return df_batchs[df_batchs.lot == batch].nom.values[0]

# normalize 'lot_0014' (from annotations) → 'Lot 14' (from users list) before matching lot & username
df_interagr['username'] = df_interagr.batch.copy().str.replace('^lot_0+', 'Lot ').apply(lambda batch: name_from_batch(batch))

CPU times: user 6.54 s, sys: 76 ms, total: 6.62 s
Wall time: 6.62 s


## Compute stats

### vectorize the types' labels

#### Split column types as a list() of values, before vectorizing them

In [28]:
df_interagr.types = df_interagr.types.str.split(' ')

In [29]:
df_interagr = df_interagr.fillna('n_a')

#### Vectorize the `types` values (multi-valued field)

In [30]:
all_types = list({t for ts in df_interagr.types for t in ts})

In [31]:
all_types.sort()
all_types

['Dipositif',
 'Dispositif-1',
 'Dispositif-2',
 'Dispositif-3',
 'Dispositif-demandes_accessoires',
 'Entete_appelant',
 'Entete_avocat',
 'Entete_composition_de_la_cour',
 'Entete_intime',
 'Expose_litige',
 'Faits_et_procedure',
 'Faits_et_procedure_faits',
 'Faits_et_procedure_procedure',
 'Motif-1',
 'Motif-1_faits',
 'Motif-1_pretentions_appelant',
 'Motif-1_pretentions_intime',
 'Motif-1_texte',
 'Motif-2',
 'Motif-2_faits',
 'Motif-2_pretentions_appelant',
 'Motif-2_pretentions_intime',
 'Motif-2_texte',
 'Motif-3',
 'Motif-3_faits',
 'Motif-3_pretentions_appelant',
 'Motif-3_pretentions_intime',
 'Motif-3_texte',
 'Motif-demandes_accessoires',
 'Motif_de_la_decision',
 'Moyens_et_pretentions',
 'Moyens_et_pretentions_appelant',
 'Moyens_et_pretentions_intime',
 'References_decision_attaquee',
 'n_a']

In [32]:
lb = preprocessing.LabelBinarizer()
lb.fit(all_types)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [33]:
lb.classes_

array(['Dipositif', 'Dispositif-1', 'Dispositif-2', 'Dispositif-3',
       'Dispositif-demandes_accessoires', 'Entete_appelant',
       'Entete_avocat', 'Entete_composition_de_la_cour', 'Entete_intime',
       'Expose_litige', 'Faits_et_procedure', 'Faits_et_procedure_faits',
       'Faits_et_procedure_procedure', 'Motif-1', 'Motif-1_faits',
       'Motif-1_pretentions_appelant', 'Motif-1_pretentions_intime',
       'Motif-1_texte', 'Motif-2', 'Motif-2_faits',
       'Motif-2_pretentions_appelant', 'Motif-2_pretentions_intime',
       'Motif-2_texte', 'Motif-3', 'Motif-3_faits',
       'Motif-3_pretentions_appelant', 'Motif-3_pretentions_intime',
       'Motif-3_texte', 'Motif-demandes_accessoires',
       'Motif_de_la_decision', 'Moyens_et_pretentions',
       'Moyens_et_pretentions_appelant', 'Moyens_et_pretentions_intime',
       'References_decision_attaquee', 'n_a'], 
      dtype='<U31')

### number of errors

In [34]:
df_interagr['vect_types'] = df_interagr.types.apply(lambda x: lb.transform(x).sum(axis=0))

In [35]:
df_interagr[:3]

,batch,file,total_line_number,line_num,types,annotation_difficulty,types_macro,sub_type,Parties,Argument,text,username,vect_types
181,lot_0037,JURITEXT000033214445.txt,60,5,[References_decision_attaquee],Facile,Entete,References_decision_attaquee,n_a,n_a,Décision déférée à la Cour : jugement du Conseil de Prud'hommes de POINTE A PITRE du 20 mai 2014...,n_a,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
182,lot_0037,JURITEXT000033214445.txt,60,6,[Entete_appelant],Facile,Entete,Entete_parties,Appelant,n_a,APPELANTE,n_a,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
183,lot_0037,JURITEXT000033214445.txt,60,7,[Entete_appelant],Facile,Entete,Entete_parties,Appelant,n_a,Madame Béatrice X...,n_a,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [36]:
df_interagr.username.unique()

array(['n_a', 'Céline Baudon', 'Eve Donnadieu', 'Camille Le Douaron',
       'Isabelle Francelet', 'Catherine B', 'Sophie Mazoyer',
       'Marie-Laure Bellamy-Brown', 'Anne Kinoo', 'Amandine Ong',
       'E Donnadieu / C Baudon / A. Ong / C.Politi', 'sandra marie laure',
       'Céline baudon', 'Bruno Mathis', 'J. Martin',
       'A. Dary / ML Bellamy-Brown', 'S. Mayet / J. Martin', 'Shathana'], dtype=object)

In [37]:
#valid_groups = df_interagr.groupby(['file', 'line_num']).filter(lambda group: len(group) >= 2).groupby(['file', 'line_num'])
valid_groups = df_interagr.groupby(['file', 'line_num'])

In [38]:
errors = valid_groups.vect_types.apply(lambda t: skmetrics.zero_one_loss(t.iloc[0], t.iloc[1], normalize=False))

In [39]:
df_errors = pd.DataFrame(errors)

In [40]:
df_errors.columns = ['nb_errors'] # needed to add a new column during the join, see below

In [41]:
df_interagr = df_interagr.join(df_errors, on=['file', 'line_num'])
del(errors)

In [42]:
df_errors.head()

nb_errors
file                     line_num           
JURITEXT000033128534.txt 11                0
                         12                0
                         13                0
                         14                0
                         15                0

In [43]:
df_errors.describe()

,nb_errors
count,6660.000000
mean,0.730480
std,0.972919
min,0.000000
25%,0.000000
50%,0.000000
75%,2.000000
max,4.000000


add the result to the DataFrame

#### Show the differences

In [44]:
print('# errors ≥ 1 for {:2.1%} of lines.'.format(df_interagr[df_interagr.nb_errors > 0].shape[0] / df_interagr.shape[0]))

# errors ≥ 1 for 36.5% of lines.


## Compute Cohen-κ by file
and for each type of annotation (multi-label)

In [45]:
# expand list of types to 1 column for each type of annotation
df_interagr[lb.classes_] = pd.DataFrame(df_interagr.vect_types.values.tolist(), index=df_interagr.index)

In [46]:
# test on 1 line

def get_cohen_k(g):
    # rows = unique files for inter agreement; row: (nb lines × nb annotators)
    nb_annotators = 2
    types_to_compare = g.reshape(nb_annotators, -1)
    ck = cohen_kappa_score(types_to_compare[0], types_to_compare[1])
    return ck

df_cohen_k = pd.DataFrame()
for t in lb.classes_[:1]:
    grp = pd.DataFrame(df_interagr.groupby(['file']).apply(lambda x: x[t].values))
    grp.columns = ['cohen_k_' + t]
    #print(grp[grp['cohen_k_' + t].apply(lambda x: not is_even(len(x)))])
    grp.applymap(lambda x: get_cohen_k(x))
    df_cohen_k = pd.concat([df_cohen_k, grp], axis=1)
    
#df_cohen_k['cohen_k_' + lb.classes_[0]]

In [47]:
def get_cohen_k(g):
    #print(f'type(g) {type(g)}, {g.shape}')
    # rows = unique files for inter agreement; row: (nb lines × nb annotators)
    nb_annotators = 2
    types_to_compare = g.reshape(nb_annotators, -1)
    #print(types_to_compare.shape)
    ck = cohen_kappa_score(types_to_compare[0], types_to_compare[1])
    return ck

df_cohen_k = pd.DataFrame()
for t in lb.classes_:
    grp = pd.Series(df_interagr.groupby(['file']).apply(lambda x: x[t].values))
    grp.name = 'cohen_k_' + t
    #print(grp.head())
    #print(f'shape grp: {grp.shape}')
    #print(f' grp: {grp[:1]}')
    ck = grp.apply(lambda x: get_cohen_k(x))
    #print(ck.head())
    df_cohen_k = pd.concat([df_cohen_k, ck], axis=1)
    

In [48]:
df_cohen_k = df_cohen_k.fillna(1) # NaN happen when the vector contains only "0", so we mark that as identical
df_cohen_k

,cohen_k_Dipositif,cohen_k_Dispositif-1,cohen_k_Dispositif-2,cohen_k_Dispositif-3,cohen_k_Dispositif-demandes_accessoires,cohen_k_Entete_appelant,cohen_k_Entete_avocat,cohen_k_Entete_composition_de_la_cour,cohen_k_Entete_intime,cohen_k_Expose_litige,...,cohen_k_Motif-3_pretentions_appelant,cohen_k_Motif-3_pretentions_intime,cohen_k_Motif-3_texte,cohen_k_Motif-demandes_accessoires,cohen_k_Motif_de_la_decision,cohen_k_Moyens_et_pretentions,cohen_k_Moyens_et_pretentions_appelant,cohen_k_Moyens_et_pretentions_intime,cohen_k_References_decision_attaquee,cohen_k_n_a
file,,,,,,,,,,,,,,,,,,,,,
JURITEXT000033128534.txt,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.000000,1.000000,0.000000
JURITEXT000033128541.txt,1.000000,1.000000,1.000000,0.853755,0.853755,1.000000,1.000000,1.000000,0.937605,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,-0.012882
JURITEXT000033128813.txt,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,1.000000,1.000000,0.000000,1.000000,1.0,1.000000,0.936119,1.000000,0.000000
JURITEXT000033128888.txt,1.000000,1.000000,1.000000,0.881579,1.000000,1.000000,0.663551,1.000000,1.000000,1.000000,...,1.000000,1.000000,0.649123,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,-0.006993
JURITEXT000033128960.txt,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.000000,1.000000,1.000000
JURITEXT000033129271.txt,1.000000,0.420063,0.575931,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,0.915718,1.000000,1.000000,0.000000
JURITEXT000033129573.txt,1.000000,1.000000,1.000000,1.000000,1.000000,0.791809,0.791809,1.000000,0.483676,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,0.848635,0.577367,1.000000,1.000000
JURITEXT000033129723.txt,1.000000,1.000000,1.000000,1.000000,1.000000,0.883774,1.000000,1.000000,0.627306,1.000000,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.0,0.679772,0.000000,1.000000,1.000000
JURITEXT000033130005.txt,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.0,1.000000,1.000000,0.000000,0.000000


---

In [49]:
df_cohen_k.describe().T

,count,mean,std,min,25%,50%,75%,max
cohen_k_Dipositif,102.0,0.759090,0.421128,-0.058824,0.745000,1.000000,1.0,1.0
cohen_k_Dispositif-1,102.0,0.578617,0.396943,0.000000,0.000000,0.657243,1.0,1.0
cohen_k_Dispositif-2,102.0,0.757735,0.402223,-0.043165,0.575931,1.000000,1.0,1.0
cohen_k_Dispositif-3,102.0,0.829884,0.349211,-0.023529,1.000000,1.000000,1.0,1.0
cohen_k_Dispositif-demandes_accessoires,102.0,0.718376,0.398336,0.000000,0.485781,1.000000,1.0,1.0
cohen_k_Entete_appelant,102.0,0.742541,0.392488,-0.172414,0.652064,1.000000,1.0,1.0
cohen_k_Entete_avocat,102.0,0.749124,0.394342,0.000000,0.645517,1.000000,1.0,1.0
cohen_k_Entete_composition_de_la_cour,102.0,0.755507,0.419242,0.000000,0.864710,1.000000,1.0,1.0
cohen_k_Entete_intime,102.0,0.737404,0.388088,-0.191083,0.676089,0.968803,1.0,1.0
cohen_k_Expose_litige,102.0,0.696168,0.462540,-0.030303,0.000000,1.000000,1.0,1.0


add the result to the DataFrame

In [50]:
df_interagr = df_interagr.join(df_cohen_k, on=['file'])
del(df_cohen_k)

In [51]:
df_interagr = df_interagr.drop(list(lb.classes_) + ['vect_types'], axis=1)

#### Save to CSV

In [52]:
#df_interagr_sorted = df_interagr[df_interagr.cohen_kappa < 1].sort_values(['file', 'line_num'])
df_interagr_sorted = df_interagr.sort_values(['file', 'line_num'])
df_interagr_sorted.columns

Index(['batch', 'file', 'total_line_number', 'line_num', 'types',
       'annotation_difficulty', 'types_macro', 'sub_type', 'Parties',
       'Argument', 'text', 'username', 'nb_errors', 'cohen_k_Dipositif',
       'cohen_k_Dispositif-1', 'cohen_k_Dispositif-2', 'cohen_k_Dispositif-3',
       'cohen_k_Dispositif-demandes_accessoires', 'cohen_k_Entete_appelant',
       'cohen_k_Entete_avocat', 'cohen_k_Entete_composition_de_la_cour',
       'cohen_k_Entete_intime', 'cohen_k_Expose_litige',
       'cohen_k_Faits_et_procedure', 'cohen_k_Faits_et_procedure_faits',
       'cohen_k_Faits_et_procedure_procedure', 'cohen_k_Motif-1',
       'cohen_k_Motif-1_faits', 'cohen_k_Motif-1_pretentions_appelant',
       'cohen_k_Motif-1_pretentions_intime', 'cohen_k_Motif-1_texte',
       'cohen_k_Motif-2', 'cohen_k_Motif-2_faits',
       'cohen_k_Motif-2_pretentions_appelant',
       'cohen_k_Motif-2_pretentions_intime', 'cohen_k_Motif-2_texte',
       'cohen_k_Motif-3', 'cohen_k_Motif-3_faits',
     

Convert types back to a list of tokens separated by a space

In [53]:
df_interagr_sorted.types = df_interagr_sorted.types.apply(lambda xs: ' '.join(xs))

In [54]:
file_result = pathlib.Path(dir_base) / 'interagr.csv'
df_interagr_sorted.to_csv(file_result
                          , index=None
                          , sep=csv_separator_out
                          , quoting=csv.QUOTE_NONNUMERIC
                          , encoding=csv_encoding)
print(f'Result in {file_result}')

Result in /home/ojeulin/developpement/data/IA-et-droit-zonage-décisions-lots/brat_6-12/interagr.csv


In [55]:
!head  -2 "$file_result"

"batch","file","total_line_number","line_num","types","annotation_difficulty","types_macro","sub_type","Parties","Argument","text","username","nb_errors","cohen_k_Dipositif","cohen_k_Dispositif-1","cohen_k_Dispositif-2","cohen_k_Dispositif-3","cohen_k_Dispositif-demandes_accessoires","cohen_k_Entete_appelant","cohen_k_Entete_avocat","cohen_k_Entete_composition_de_la_cour","cohen_k_Entete_intime","cohen_k_Expose_litige","cohen_k_Faits_et_procedure","cohen_k_Faits_et_procedure_faits","cohen_k_Faits_et_procedure_procedure","cohen_k_Motif-1","cohen_k_Motif-1_faits","cohen_k_Motif-1_pretentions_appelant","cohen_k_Motif-1_pretentions_intime","cohen_k_Motif-1_texte","cohen_k_Motif-2","cohen_k_Motif-2_faits","cohen_k_Motif-2_pretentions_appelant","cohen_k_Motif-2_pretentions_intime","cohen_k_Motif-2_texte","cohen_k_Motif-3","cohen_k_Motif-3_faits","cohen_k_Motif-3_pretentions_appelant","cohen_k_Motif-3_pretentions_intime","cohen_k_Motif-3_texte","cohen_k_Motif-demandes_accessoires","cohen_k_Mo